# Loans Data Exploration

## 1. Preliminary Wrangling
This document explores a dataset containing metadata related to loans such as: 
- The type of loan, duration, amount, etc, to
- Customers employment information that would impact there borrowing power i.e. credit rating, job type, etc. <br>
`Additional variables provided in the data dictionary, Prosper Loan Data`

### 1.1. Initialize

#### 1.1.1 Install packages

In [ ]:
#pip install <package> via windows
#!pip install or !apt-get install for google colab
#!pip install modin

#### 1.1.2 Import libraries

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
#import modin.pandas as pd
import pandas as pd
import matplotlib.ticker as ticker

%matplotlib inline

### 1.2. Load dataset

#### 1.2.1. Google Colab

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# #Google colab read csv
# loans_raw = pd.read_csv('/content/drive/My Drive/Colab Notebooks/prosperLoanData.csv')

#### 1.2.2. Local **folder**

In [ ]:
# Local drive read csv
loans_raw = pd.read_csv('prosperLoanData.csv')

1.2.3 Copy Raw Dataframe

In [ ]:
# reload loans_df with the relevant columns, loans_raw contains the original
loans_df = loans_raw.copy()
loans_df.head()

### 1.3. GLOBAL

#### 1.3.1 Functions

In [ ]:
# FUNCTION TO PLOT VALS ALONG X-AXIS, NOMINAL VALS ON Y
def yplot_values(gx, form):
    initialx = 0

    for g in gx.patches:
        gx.text(g.get_width(), initialx + g.get_height()/4,
                form.format(g.get_width()),
                color='black',
                ha="left") 
        initialx+=1

In [ ]:
# FUNCTION TO PLOT VALS ALONG Y-AXIS, NOMINAL VALS ON X
def xplot_values(gy, form):
    initialy = 0
    
    # Logic to print the proportion text on the bars
    for g in gy.patches:
        gy.text(initialy + g.get_width()/13, g.get_height(), 
        form.format(g.get_height()), 
        color='black',
        ha='center', # 'center', 'right', 'left'
        va='bottom') # 'top', 'bottom', 'center', 'baseline', 'center_baseline'
        initialy+=1

### 1.4.1. Data Structure
There are 113,937 entries, across 81 variables (as seen above) in this loans dataset.
Below are the variables of interest during this exploration.

In [ ]:
loans_raw.shape # review shape of dataset

In [ ]:
loans_raw.info()

In [ ]:
# create list
subset = ['EmploymentStatus', 'BorrowerState', 'Occupation', 
'CreditGrade', 'ProsperRating (Alpha)', 'ProsperRating (numeric)',
'IncomeRange', 'Term', 'CurrentDelinquencies', 'BorrowerRate', 'DebtToIncomeRatio', 
'ListingCreationDate', 'ClosedDate', 'MonthlyLoanPayment']

subset, print('Number of Target Columns: {}\n'.format(len(subset)))

#### 1.4.1 Duplicates

In [ ]:
## Check for duplicate data
loans_raw[subset].duplicated().value_counts()

### 1.5. Data Cleaning

#### 1.5.1. Data dictionary variables of interest Summary:
Before continuing any further only columns of interest will be retained for the exploration to ensure only the required variables are modified.

In [ ]:
# size before modification
loans_df.shape

In [ ]:
# reload loans_df with the relevant columns, loans_raw contains the original
loans_df = loans_raw.loc[:, subset].copy()
loans_df.head()

#### 1.5.2. Datatypes

##### 1.5.2.1 Date time correction

In [ ]:
# convert to correct data types
## dates
toDates = ['ListingCreationDate', 'ClosedDate'] # date mask
loans_df[toDates] = loans_df[toDates].astype('datetime64')

In [ ]:
# check datatypes are now correct
loans_df[toDates].dtypes

##### 1.5.2.2 Date Year and Month extraction

In [ ]:
# Extract year and month as ProsperRating and CreditGrade vary with year and month
loans_df['ListingCreationYear'] = pd.DatetimeIndex(loans_df.ListingCreationDate).year
loans_df['ListingCreationMonth'] = pd.DatetimeIndex(loans_df.ListingCreationDate).month
loans_df['ClosedDateYear'] = pd.DatetimeIndex(loans_df.ClosedDate).year

In [ ]:
loans_df.info()

##### 1.5.2.4 Combine pre and post 2009 ratings
There are ratings we were recorded pre 2009 and post 2009.

**CreditGrade** and **ProsperRating**, these two are pairs because they cannot be analysed separately due to each rating type being based on a specific time period, where the former was used for users _pre 2009_ and the latter was for _post 2009_.
Quickly reviewing the two columns reveals there are **Nan** evident on either field whilst the other contains an entry.
As such the dataframe will contain a new column combining the two, specifically the _post 2009_ overwritting the *Nan* values.

In [ ]:
# combine pre 2009 followed by post 2009
loans_df['CreditRating'] = loans_df['CreditGrade'].fillna(loans_df['ProsperRating (Alpha)'])

A dictionary will be created to assign and apply the relevant numerical values to the

In [ ]:
# create a dictionary to assign key value pairs
ratings_dict = {
    'AA':7,
    'A':6,
    'B':5,
    'C':4,
    'D':3,
    'E':2,
    'HR':1,
    'Nan':None,
}

In [ ]:
# assign value to new column
loans_df['ProsperRatingNum'] = loans_df['CreditRating'].map(ratings_dict)

In [ ]:
loans_df['ProsperRatingNum'].dtypes

In [ ]:
loans_df['ProsperRatingNum'] = pd.to_numeric(loans_df['ProsperRatingNum'], downcast='float', errors='coerce')

In [ ]:
loans_df['ProsperRatingNum'].dtypes

In [ ]:
# filter employmentstatus and income range
loans_df = loans_df[(loans_df.EmploymentStatus!='Not available')& (loans_df.IncomeRange!='Not displayed')]

Having **Not Available** and **Income Displayed** is redundant information that doesn't assist in the analysis of what affects BorrowRate.
<br>
With the **Employment Status** analysed, the next breakdown from Employment are the **Occupations**.

##### 1.5.2.3 Category re-definition

Income range requires cleaning as evident with **\$0** and **Not employed** values.

In [ ]:
loans_df.IncomeRange.value_counts(dropna=False)

In [ ]:
loans_df.IncomeRange = loans_df.IncomeRange.replace(['Not employed'], '$0') # replace incorrect data
loans_df.IncomeRange = loans_df.IncomeRange.replace(['Not displayed'], None)

In [ ]:
# Find ordinal variables and set order from lowest (left) to highest (right)
ordinal_dict = {'CreditGrade': ['HR','E','D','C','B','A','AA'],
                'CreditRating': ['HR','E','D','C','B','A','AA'],
                'IncomeRange': ['$0', '$1-24,999', '$25,000-49,999', '$50,000-74,999', '$75,000-99,999', '$100,000+']
                }

# Udacity code snippet to replace Column in existing DF with newly configured order (from above)
for var in ordinal_dict:
    ordered_var = pd.api.types.CategoricalDtype(ordered = True,
                  categories = ordinal_dict[var])
    
    loans_df[var] = loans_df[var].astype(ordered_var)

In [ ]:
loans_df['CreditGrade'].dtypes

In [ ]:
loans_df['IncomeRange'].dtypes

##### 1.5.2.5 Column type definitions

In [ ]:
# separation of variables
numbers = ['ProsperRatingNum', 'Term', 'CurrentDelinquencies', 'BorrowerRate', 'DebtToIncomeRatio', 'MonthlyLoanPayment']
categories = ['EmploymentStatus','BorrowerState', 'Occupation', 'CreditRating', 'CreditGrade', 'IncomeRange']

### 1.6. Univariate Exploration

#### 1.6.1 NaN/Missing Data
First visual is to grasp the amount of NaN present in the current dataset to determine whether additional cleaning is required. 

In [ ]:
# create empty list to hold series data, information is held in .index (column names) and .values (number of NaN per column)
Nan_sublist_all = []
Nan_sublist_all = loans_df.isna().sum()
Nan_sublist_all.sort_values(ascending=False)

In [ ]:
# List above contains unnecessary entries limiting viewing.
# Add equality test to narrow down to column names with 
Nan_sublist = Nan_sublist_all[Nan_sublist_all.values > 0]

In [ ]:
Nan_sublist.shape

In [ ]:
# explicit figure and figsize declaraction
fig, axes = plt.subplots(1,1)
sb.despine(fig)

# plot missing values in descending order
g0 = sb.barplot(x=Nan_sublist.values, y=Nan_sublist.index, ax=axes, color='red', order=Nan_sublist.sort_values(ascending=False).index )

yplot_values(g0, '{:1.0f}'); # plot y values onto graph
fig.suptitle('Number of NA values in each Variable', fontsize=18);

In [ ]:
loans_df.shape # review size prior to dropping Nan

In [ ]:
Nan_sublist.sort_values(ascending=False)[4:].index

In [ ]:
# slice object, convert to list and check datatype
subset_drop = Nan_sublist.sort_values(ascending=False)[4:].index.tolist()
subset_drop, type(subset_drop)

In [ ]:
# drop Nan values
loans_df.dropna(axis=0, subset=subset_drop, inplace=True)

In [ ]:
loans_df.shape

## 2 Exploratory data analysis
The EDA will revolve around the exploration of the 14 variables mentioned previously.

In [ ]:
loans_df.info()

#### 2.1. Numerical variables

In [ ]:
# drop CreditGrade as it creates an additional row/column in the pair plot
loans_df.drop(labels='CreditGrade', axis=1, inplace=True)

In [ ]:
# set 'n' samples 
loans_samples = loans_df.sample(n=1000)

In [ ]:
# pairplot
g0 = sb.pairplot(loans_samples, kind='scatter', dropna=True, height=2.5);

g0.fig.subplots_adjust(top=0.9)
g0.fig.suptitle('Comparison of Numerical Variables', fontsize=28);

Of the 500 random samples:<br>
- **Prosper score** shows a normal distribution, with a strong negative correlation to **BorrowRate** and a weak positive correlation with **MonthlyLoanPayment**.
- **Term** only is left skewed, with 3 main values.
- **CurrentDelinquencies** is right skewed, with possibly a weak positive correlation to **BorrowRate** and a weak negative correlation to **MonthlyLoanRepayment**.
- **BorrowRate** appears bimodal.
- **DebtIncome** appears heavily right skewed with a large peak of values around 0.
- **MonthlyLoanPayment** is right skewed.

Of the quantitative variables above, we will investigate **Prosper Score/Credit Grade, Borrowrate, MonthlyLoanPayment** to clarify the correlations seen above against categorical variables that have not yet been considered.
The heatmap below provides numeric evidence of the visual inspection of the pair grid.

In [ ]:
# heatmap
g1 = sb.heatmap(loans_df.corr(), annot = True, fmt = '.2f',
           cmap = 'vlag_r', center = 0);

plt.title('Numerical Correlations', fontsize=20);

### 2.2 What affects a loan repayments interest rate/BorrowRate?
What factors greatly affect the BorrowRate and as a result the **MonthlyLoanPayment** of the customer?

#### 2.2.1. Borrower Rate

In [ ]:
loans_df.BorrowerRate.describe().reset_index()

In [ ]:
# initialize
fig, axes = plt.subplots(1,1, figsize=(10, 4))
sb.despine(fig)

# set ticks
minTickBorrow = np.arange(0, loans_df.BorrowerRate.max()+0.04, 0.02)

# graph 1
g0 = sb.violinplot(x=loans_df.BorrowerRate, color='Red', ax=axes) # graph 1
plt.setp(g0.collections, alpha=.6)

# graph 2
g0 = sb.boxplot(x=loans_df.BorrowerRate, boxprops=dict(alpha=.8), ax=axes) # graph 2
g0.set(xticks=minTickBorrow, xlabel='Borrow Rate %')

fig.suptitle('Borrower Rate Data Spread', fontsize=20);

Both graphs approximate the median **BorrowRate** to be 18%, both plots suggest outliers to be considered ~38% and above. This allows unless to optimize the bins to be between 0 and 0.4.

In [ ]:
#initialize
fig, axes = plt.subplots(2,1, figsize=(10,6), sharex=True)
sb.despine(fig)

# graph 2 - distribution of borrow rates
g1 = sb.histplot(loans_df, x='BorrowerRate', element='bars', multiple="stack", 
        ax=axes[0]);

# graph 3
axes[1].set(yscale="log") # scale y axis only
g2 = sb.histplot(loans_df, x='BorrowerRate', element='bars', multiple="stack",
        #log_scale=True, # log scales both x & y
        ax=axes[1]);
g2.set(xticks=minTickBorrow, xlabel='Borrow Rate %')

fig.suptitle('Borrower Rate Frequency', fontsize=16); # set title

**BorrowRate** variable shows a multi-modal normal distribution, one towards the lower end where the bulk of the loaners are within 10% to 20%. The second towards the higher end with a peak at 31%.

The log scale transformation reveals the outliers not seen in the prior 2 graphs above it.

With the distribution of primary variable explored. The remaining categorical variables that were not able to be analysed in the pair plot will be investigated to determine possibly influences with **BorrowerRate**.

#### 2.2.2. Employment Status

In [ ]:
loans_df.EmploymentStatus.value_counts()

In [ ]:
# initialize
fig, axes = plt.subplots(3,1, figsize=(16,12))
sb.despine(fig)

# graph 1 - employment status
g0 = sb.barplot(x=loans_df.EmploymentStatus.value_counts().index,
        y=loans_df.EmploymentStatus.value_counts().values, ax=axes[0]);
xplot_values(g0, '{:1.0f}') # call func. add values to the chart

# graph 2 - distribution of borrow rates
g1 = sb.histplot(loans_df, x='BorrowerRate', binrange=(0, 0.4), element='bars', hue='EmploymentStatus', multiple='stack', ax=axes[1]);
g1.set(xticks = minTickBorrow, xlabel='Borrow Rate %', 
        yticks= np.arange(0, 5000, 500))

# graph 3 - log scale transformation of of graph 1 to assist in visualizing smaller values
axes[2].set(yscale="log")
g2 = sb.histplot(loans_df, x='BorrowerRate', binrange=(0, 0.4), element='bars', hue='EmploymentStatus', multiple='stack', ax=axes[2], legend=False);
g2.set(xticks = minTickBorrow, xlabel='Borrow Rate %')

plt.suptitle('BorrowerRate vs Employment', fontsize=20);

The univariate exploration of **EmploymentStatus** reveals majority of the loan applicants are 'Employed', which is redundant as it splits further into _Full-time Employed, Self-Employed, Part-time_, other etc meaning the data classifications were not considered appropriately.
The Bivariate exploration against **BorrowRate**, reveals the spread of employment with generally revolving around 10 to 30% +/- 5%, with a large peak at ~32%.
The log transformation provides clarity for the lower values not otherwise seen in graph 2, the borrow rate of _full-time_ employees receiving ~0.4% is interesting. If they were incorrectly entered or due to the low overall amount required to be borrowed that led to such a low rate.

In [ ]:
# initialize
sb.set_style("whitegrid", {'grid.linestyle': '--'})
fig, axes = plt.subplots(1,1, figsize=(12, 7))
sb.despine(fig)

# prepare ticks
minTickBorrow = np.arange(0, loans_df.BorrowerRate.max()+0.1, 0.02)

# graph 4
g3 = sb.violinplot(x=loans_df.BorrowerRate, y=loans_df.EmploymentStatus, ax=axes);
plt.setp(g3.collections, alpha=.8)

# graph 5
g4 = sb.boxplot(x=loans_df.BorrowerRate, y=loans_df.EmploymentStatus, ax=axes);
g4.set(xticks = minTickBorrow, xlabel='Borrow Rate %');

# filter out employed and other
employment_mask=~loans_df['EmploymentStatus'].isin(['Employed', 'Other']) 

fig.suptitle('Employment Status', fontsize=20);

It is evident that **BorrowerRate** is affected by **EmploymentStatus** given the following:
-Of the 7 employment types, the median borrow rate of **Not employed** is significantly higher then the remaining, as well a large majority of it's spread situated at the higher borrow rates due to the increased risk of the individual paying it back.
-**Self-employed** & **Full-time** employees having the lowest medians
-**Employed** & **Other** are generalised as discussed previously.

It is interesting to see **Other** has the widest spread at ~32%, with **Employed** having the second highest width. The remaining employment types seem to taper off at the same amount.

#### 2.2.3. Occupation

In [ ]:
loans_df.Occupation.value_counts()

In [ ]:
# subplots(row, col) , figsize(x,y)
fig, axes = plt.subplots(1,1, figsize=(7,12))
sb.despine(fig)

#axes.set(xscale="log")
g0 = sb.countplot(data=loans_df, y='Occupation',
                  order=loans_df.Occupation.value_counts().index);

# call function and plot values of graph
yplot_values(g0, '{:1.0f}')

fig.suptitle('Occupation Counts');

In [ ]:
# occupation mask

The above graph shows several types of Occupations, split into there appropriate position name. It is important to note the top two occupations are ambigious and uninformative i.e. **Professionals** and **Other**.

As there several Occupations to display, it will be quite cumbersome to display all the statistical distributions of each, example seen below. 

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(9, 12))
sb.despine(fig)

g1 = sb.boxplot(y= loans_df.Occupation, x = loans_df.BorrowerRate);
g1.set(xticks = np.arange(0, loans_df['BorrowerRate'].max()+0.04, 0.02), xlabel='Borrow Rate %'); #preset variables

fig.subplots_adjust(top=0.9) # adjust offset
fig.suptitle('BorrowerRate vs Occupation', fontsize=20);

Using the box plot above, we can see a better capture of the **Occupations** relationship to **BorrowerRate**. It is particularly evident that **Occupations** do impact borrow rates as they are a subset of **EmploymentStatus** evident with  **Judge** and **Doctors** appearing to have the lowest median borrow rates, where as **Student College Freshman** and **Teachers aid** appear to have the highest.

Due to the quantity of the **Occupations** list, a select few professions will be assessed to ensure a wide proportion of the population is captured. This selection is visual based with attempts to collect a low and high borrow rate of each field i.e. business, law, engineering, public service, admin, etc.

In [ ]:
# initialize
fig, axes = plt.subplots(4, 1, figsize=(10, 8), sharex=True)
sb.despine(fig)

# filter by text
g2 = sb.boxplot( y= loans_df[loans_df['Occupation'].str.contains(pat='Trade',regex=False)].Occupation,
  x = loans_df.BorrowerRate, ax=axes[0]);

g3 = sb.boxplot(y= loans_df[loans_df['Occupation'].str.contains(pat='Student',regex=False)].Occupation,
  x = loans_df.BorrowerRate, ax=axes[1]);

g4 = sb.boxplot(y= loans_df[loans_df['Occupation'].str.contains(pat='Engineer',regex=False)].Occupation,
  x = loans_df.BorrowerRate, ax=axes[2]);

g5 = sb.boxplot(y= loans_df[loans_df['Occupation'].str.contains(pat='Retail',regex=False)].Occupation,
  x = loans_df.BorrowerRate, ax=axes[3]);
g5.set(xticks = minTickBorrow, xlabel='Borrow Rate %');

fig.suptitle('Sample Occupations vs BorrowerRate', fontsize=20);

In [ ]:
# sliced list omitting Professional and Other
key_occp_list = loans_df.Occupation.value_counts().index[2:11].to_list()

# custom selection of Occupations to incorporate one from each field of work
custom_occp_list = ['Police Officer/Correction Officer', 'Fireman', 
                    'Attorney', 'Engineer - Mechanical', 
                    'Doctor', 'Judge', 'Investor', 'Sales - Retail',
                    'Student - College Freshman', 'Student - College Graduate Student']

sample_occp_list = key_occp_list + custom_occp_list # append both lists into one
sample_occp_mask = loans_df.Occupation.isin(sample_occp_list)

In [ ]:
# initialize
fig, axes = plt.subplots(1,1, figsize=(12,10))
sb.despine(fig)

g6 = sb.violinplot(
        y = loans_df[sample_occp_mask].Occupation,
        x = loans_df.BorrowerRate);
plt.setp(g6.collections, alpha=.7);

g7 = sb.boxplot(
        y = loans_df[sample_occp_mask].Occupation,
        x = loans_df.BorrowerRate,
        boxprops=dict(alpha=.8));
g7.set(xticks = np.arange(0, loans_df.BorrowerRate.max()+0.1, 0.02), xlabel='Borrow Rate %')

fig.suptitle('Sample list of Occupations vs BorrowerRate', fontsize=20);

Initially the two plots were on separate subplots which made comparing them side by side difficult. As such combining them onto the same plot allows for a better visual of the width of the violins, as well as the outliers, lower and upper quartiles shown by the box plot. It is apparent **Occupations** do impact the borrow rates issued out by Prosper, next variable to investigate is **Income Range** which directly ties to the amount each occupation is paid.


Prior to dropping **Other** and **Professional** occupations, an investigation into the spread of data will be performed to determine how it sits when compared against **Borrower Rate**.

In [ ]:
loans_df.shape # taking note of current dataframe size

In [ ]:
loans_df[sample_occp_mask].shape # the size based on the above filter

In [ ]:
# the size based on Other and Professional
sample_misc_loans = loans_df[loans_df.Occupation.isin(['Other', 'Professional'])] # get Occupations with Other and Professional
sample_misc_loans.shape

In [ ]:
# the size of the remaining excluding other and Professional
sample_remain_loans = loans_df[(loans_df.Occupation!='Other') & (loans_df.Occupation!='Professional')]
sample_remain_loans.shape

In [ ]:
# plot to show the distribution of Other and Professional occupations
# subplots(row, col)
fig, axes = plt.subplots(2,1, figsize=(10,6), sharex=True)
sb.despine(fig)

# graph 1 - distribution of borrow rates
g8 = sb.histplot(sample_misc_loans, # new sample data frame above
        x='BorrowerRate', binrange=(0, 0.4), element='bars',
        hue='Occupation', multiple='stack', ax=axes[0]);

# graph 2 - log scale transformation of of graph 1 to assist in visualizing smaller values
axes[1].set(yscale="log")
g9 = sb.histplot(sample_misc_loans, 
        x='BorrowerRate', binrange=(0, 0.4), element='bars',
        hue='Occupation', multiple='stack', ax=axes[1], legend=False);
g9.set(xticks = np.arange(0, loans_df['BorrowerRate'].max()+0.04, 0.02), xlabel='Borrow Rate %')

# TODO - consolidate legend
fig.suptitle('BorrowRate for Other and Professional Occupations', fontsize=20);

Before dropping **Other** and **Professional** occupations from the dataframe, we observe the plot above showing a normal distribution and its potential influence to BorrowRate. The peak at ~32% would appear to the usual BorrowRate provided by Prosper.

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(10,3))
sb.despine(fig)

g0 = sb.boxplot(data = sample_misc_loans,
                x = 'BorrowerRate', y = 'EmploymentStatus', hue='Occupation');
g0.set(xticks = np.arange(0, loans_df.BorrowerRate.max()+0.04, 0.02), xlabel='Borrow Rate %')
g0.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

fig.suptitle('BorrowRate for "Other and Professional Occupations"')

#### 2.2.4. IncomeRange

The next category of interest is the Incomes of the respective Occupations above as well its overall spread.

In [ ]:
loans_df.IncomeRange.value_counts(dropna=False)

In [ ]:
loans_df.ProsperRatingNum.dtypes

In [ ]:
# initialize
fig, axes = plt.subplots(2,1, figsize=(12, 12) )
sb.despine(fig)

# graph 1 - distribution of Income range
g0 = sb.histplot(data = loans_df, # new sample data frame above
    x='IncomeRange', element='bars', multiple='stack', ax=axes[0]);
g0.set(yticks=np.arange(0, 30000, 2000))
xplot_values(g0, '{:1.0f}');

# graph 2 - distribution of Income range
g1 = sb.violinplot(data = loans_df,
    x='BorrowerRate', y='IncomeRange', ax=axes[1]);
plt.setp(g1.collections, alpha=0.7);

# graph 3
g1 = sb.boxplot(data = loans_df,
    x='BorrowerRate', y='IncomeRange', boxprops=dict(alpha=0.8), ax=axes[1]);
g1.set(xticks = minTickBorrow, xlabel='Borrow Rate %');

plt.suptitle('Income Range', fontsize=20);

From the plots above we can see a left skewed distribution, with the majority of incomes ranging almost equally between **\$25,000-49,999** and **\$50,000-75,000** .

The plot of **BorrowerRate** against **IncomeRange** reveals a pattern reinforcing the idea that Income affects the **BorrowerRate** provided by Prosper, explicitly that the higher your income guarantees you will have a lower borrow rate on your loan.

In [ ]:
Occ_order=['Computer Programmer', 'Analyst',
  'Executive', 'Investor', 'Sales - Commission',
  'Accountant/CPA', 'Clerical', 'Administrative Assistant',
  'Teacher', 'Student - College Freshman', 'Student - College Graduate Student',
  'Judge', 'Attorney', 'Doctor', 'Engineer - Mechanical',
  'Fireman', 'Police Officer/Correction Officer', 'Skilled Labor']

# re-use previous list of Occupations to produce a multi-variate plot 

g0 = sb.FacetGrid(data=loans_df[sample_occp_mask], 
  # number of columns before starting a new row
  col = 'Occupation', col_wrap = 3, col_order = Occ_order,
  sharex=False, height=3, aspect=2.5, xlim=(0, 0.4),
  margin_titles = True);

g0.map_dataframe(sb.boxplot, x='BorrowerRate', y='IncomeRange');
g0.set(xticks=np.arange(0, loans_df.BorrowerRate.max()+0.04, 0.02), xlabel='Borrow Rate %' );

g0.fig.subplots_adjust(top=0.9) # adjust offset
plt.suptitle('Occupations - BorrowerRate vs IncomeRange', fontsize=20);

From the multi-variates, it is obvious that the higher income ranges, generally results in lower **BorrowerRates**.
The only exception to these are **Doctors**, **Clerical**, **Investors** and **Student - College Graduate Students**. 
It is interesting to see how there are some loans approved despite being listed for having **No income** otherwise considered as **\$0** as shown above. 

#### 2.2.5. Monthly Loan Payment & Term

In [ ]:
loans_df.Term.value_counts()

In [ ]:
# initialize
fig, axes = plt.subplots(2,1, figsize=(12,7), sharex=True)
sb.despine(fig)

# set ticks
mintickMLP = np.arange(0, loans_df.MonthlyLoanPayment.max()+200, 200)

# graph 1 - distribution of borrow rates
g0 = sb.histplot(loans_df, x='MonthlyLoanPayment', element='bars', multiple="stack",
        ax=axes[0]);

# graph 2
axes[1].set(yscale="log")
g1 = sb.histplot(loans_df,
        x='MonthlyLoanPayment', element='bars', multiple="stack",
        ax=axes[1]);
g1.set(xticks = mintickMLP, xlabel='Monthly Loan Payment ($)');

plt.suptitle('MonthlyLoanPayment', fontsize=20);

From the above histogram plots we are able to see a heavily right tailed distribution, where the logarithmic scaling reveals the outliers ~\$1500 per month and above, the bulk of the customer repayments ranging around \$150 per month.

In [ ]:
# set figure to have major grid lines
sb.set_style("whitegrid", {'grid.linestyle': '--'}) # provide gridlines at major ticks
fig, axes = plt.subplots(1, 2, figsize=(14,10) )
sb.despine(fig)

g0 = sb.boxplot(x=loans_df['MonthlyLoanPayment'], y=loans_df[sample_occp_mask].Occupation, # sample list of occupations
        hue=loans_df['Term'], ax=axes[0]);
g0.set(xticks = mintickMLP, xlabel='Monthly Loan Payment ($)');
       
g1 = sb.violinplot(y=loans_df['MonthlyLoanPayment'], x=loans_df.Term, ax=axes[1]);

g1 = sb.boxplot(y=loans_df['MonthlyLoanPayment'],
        x=loans_df.Term, ax=axes[1]);
g1.set(yticks = mintickMLP, ylabel='Monthly Loan Payment ($)');

fig.suptitle('Occupations - Monthly Repayment vs Term', fontsize=20);

The table summary, violin & box plot above reveals the majority of Prosper clients requiring loans prefer _36 month/3 year_ terms. 
The plot reveals a significant quantity of outliers within _12 month/1 year_ terms, afterwards the outliers gradually decrease as the terms increase. The median monthly repayments range between as low as \$100 to as high as \$400 across all **Occupations**.
<br>
<br>
Delving deeper into the spread within each of the sample occupations mentioned earlier, we can see that **Doctors** have a large IQR _(interquartile range)_ indicating a large population of doctors have loan repayments of substantial variance when taking out a _12 month/1 year_ loan, potentially indicating the professions potential buying power, spending habits/capital investments required as part of there profession, i.e. dental equipment amongst other various factors speculating if these **Doctors** start their own practice and need the large capital investment.
<br>
<br>
The graph below is the previously plotted graph as seen on the pair plot at the start of __section 2. EDA__, however this graph includes the term variable to reveal the distribution of repayments amounts based on the terms chosen.
The plot below further supports the statement above where the people prefer the _36 month/3 year_ term, it reveals the spread of individuals with there respective loan repayments vs the borrower rate they have been given by Prosper. The _36 month/3 year_ term is significantly dominant soon followed by the _60 month/5 year_ then _12 month/1 year_.

In [ ]:
loans_df.head(5)

In [ ]:
g0 = sb.jointplot(data=loans_df, x='BorrowerRate', y='MonthlyLoanPayment', hue='Term', 
  palette=['red','green','blue'], alpha=0.75, height=10, ratio=4, xlim=(0, 0.38), ylim=(-50, 2400));

g0.set_axis_labels(xlabel='Borrow Rate %', ylabel='MonthlyLoanPayment ($)')

# jointplot tick increments
g0.ax_joint.xaxis.set_major_locator(ticker.MultipleLocator(0.02))
g0.ax_joint.yaxis.set_major_locator(ticker.MultipleLocator(200))

g0.fig.subplots_adjust(top=0.95) # adjust offset
g0.fig.suptitle('Repayments - BorrowerRate vs Term', fontsize=20);

#### 2.2.6. CreditRating
CreditRating is a combination of ProsperRating & CreditGrade

In [ ]:
loans_df['CreditRating'].value_counts(dropna=False)

In [ ]:
# define figure and num. axes
sb.set_style("whitegrid", {'grid.linestyle': '--'})
sb.despine(fig)

g0 = sb.jointplot(data=loans_df, x='BorrowerRate', y='MonthlyLoanPayment', hue=loans_df['CreditRating'], hue_order=ordinal_dict['CreditGrade'],
  kind='scatter', height=10, ratio=5, space=0.3, xlim=(0, 0.38), ylim=(-50, 2400) );

g0.set_axis_labels(xlabel='Borrow Rate %', ylabel='MonthlyLoanPayment ($)')

# change x/y axis values
g0.ax_joint.xaxis.set_major_locator(ticker.MultipleLocator(0.02))
g0.ax_joint.yaxis.set_major_locator(ticker.MultipleLocator(200));

g0.fig.subplots_adjust(top=0.9) # adjust offset
g0.fig.suptitle('CreditRating - BorrowerRate vs Term', fontsize=20);

The above **joint plot** supports the ordinal data hierarchy evident with an **AA rating** receiving the lowest **BorrowerRates** and HR i.e. high risk ratings receiving the highest. As highlighted during the plotting of the numerical variables, there is a high correlation between **ProsperRating (numeric)** i.e. a numerical rating of **CreditGrade** and **ProsperRating (Alpha)**.

The bivariate **kde plot** of **MonthlyLoanPayments** is difficult to view and is plotted below, showing similar distributions for _A, B & C_ ratings. 
All the graphs apart from the _HR_ graph are shown to have right tailed distributions supporting the previous findings of clients having lower monthly repayments. The only rating that is not aligned with the others is the **HR rating**.

In [ ]:
sb.set_style("whitegrid", {'grid.linestyle': '--'})

g0 = sb.FacetGrid(data=loans_df[sample_occp_mask], col='CreditRating', col_wrap=3, 
                  col_order=ordinal_dict['CreditGrade'], sharex=False, xlim=(0,1300),
                  height=5, aspect=1.5)

g0.map(sb.histplot, 'MonthlyLoanPayment', kde=True);
g0.set(xticks=np.arange(0, 1400, 100), xlabel='Monthly Loan Payment ($)', # values trimmed from 2000 to 1000
       yticks=np.arange(0, 1100, 50)); # prior to filter 2000 was the recorded max

g0.fig.subplots_adjust(top=0.9) # adjust offset
plt.suptitle('Monthly Loan Payment Distribution of each Rating', fontsize=20);

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(10, 10))
sb.despine(fig)

# Using the graph from the 
g0 = sb.violinplot(x = loans_df['ProsperRating (numeric)'],
  y = loans_df[sample_occp_mask].Occupation);

g0 = sb.boxplot(x = loans_df['ProsperRating (numeric)'],
  y = loans_df[sample_occp_mask].Occupation);

plt.setp(g0.collections, alpha=0.7); # seaborn change transparency
g0.set(xticks = np.arange(0, 10, 1));

fig.subplots_adjust(top=0.9) # adjust offset
fig.suptitle('', fontsize=20);

In [ ]:
Occ_order=['Computer Programmer', 'Analyst',
 'Executive', 'Investor', 'Sales - Commission',
 'Accountant/CPA', 'Clerical', 'Administrative Assistant',
 'Teacher', 'Student - College Freshman', 'Student - College Graduate Student',
 'Judge', 'Attorney', 'Doctor', 'Engineer - Mechanical',
 'Fireman', 'Police Officer/Correction Officer', 'Skilled Labor']

g0 = sb.FacetGrid(data=loans_df[sample_occp_mask], col='Occupation', 
  col_wrap=4, col_order = Occ_order, #height=2,
  aspect=1.5, sharex=False, margin_titles=True);
g0.map_dataframe(sb.boxplot, 'ProsperRatingNum', 'EmploymentStatus');



In the above occupations we can see the Prosper Ratings ranging between 3 and 5, which corresponds to a B, C and D rating.
Doctors have the same IQR across the various employment status, **Employed Judges** has best box plot across across all occupations and employment types.

#### 2.2.7. BorrowerState

In [ ]:
loans_df.BorrowerState.value_counts().reset_index()

As borrower states are numerous only the top 6 states will be analysed.

In [ ]:
# top X states, 9 picked as default as it provides a 3x3 grid
states=9 # user to change

StateTopN = loans_df.BorrowerState.value_counts().head(states)
StateTopN.reset_index()

In [ ]:
state_N_mask = loans_df['BorrowerState'].isin(StateTopN.index)

In [ ]:
loans_df[state_N_mask].sample(5)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(5,3))
sb.despine(fig)

g0 = sb.histplot(loans_df[state_N_mask],
        x='BorrowerState', ax=axes);
xplot_values(g0, '{:1.0f}')

In [ ]:
g0 = sb.FacetGrid(data=loans_df[state_N_mask], col='BorrowerState', 
                  col_wrap=3, # number of columns before starting a new row
                  height=5,
                  aspect=1.5, sharex=False, margin_titles=True);
g0.map_dataframe(sb.histplot, 'BorrowerRate', kde=True);

g0.set(xticks=np.arange(0, loans_df['BorrowerRate'].max()+.04, 0.02), xlabel='Borrow Rate %')

g0.fig.subplots_adjust(top=0.9) # adjust offset
plt.suptitle('BorrowerState vs State', fontsize=20);

In [ ]:
sb.set_style("whitegrid", {'grid.linestyle': '--'})

g1 = sb.FacetGrid(data=loans_df[state_N_mask], col='BorrowerState', 
                  col_wrap=3, # number of columns before starting a new row
                  height=3,
                  aspect=2, sharex=False, margin_titles=True, xlim=(0, 1600));
g1.map_dataframe(sb.histplot, 'MonthlyLoanPayment', kde=True);
g1.set(xticks=np.arange(0, loans_df.MonthlyLoanPayment.max()+200, 200). xlabel='Monthly Loan Payment ($)' );

# figure title format
g1.fig.subplots_adjust(top=0.9)
g1.fig.suptitle('Top 9 States Monthly Loan Repayment', fontsize=20);

Of the top 9 states requiring loans, CA appears to have requested the most loans with VA being the lowest of the 9 states.

#### 2.2.8. Debt to Income Ratio

Analysis of numerical variable against categoricals

In [ ]:
loans_df.DebtToIncomeRatio.value_counts()

In [ ]:
categories

In [ ]:
sb.set_style("whitegrid", {'grid.linestyle': '--'})

# subplots(row, col)
fig, axes = plt.subplots(2,1, figsize=(8,4))
sb.despine(fig)

ticksDIR=np.arange(0, loans_df.DebtToIncomeRatio.max()+1, 1)
# graph 1
g0 = sb.histplot(data=loans_df, x='DebtToIncomeRatio', ax=axes[0]);
g0.set(xticks=ticksDIR);

# graph 2
axes[1].set(yscale="log")
g1 = sb.histplot(data=loans_df, x='DebtToIncomeRatio', ax=axes[1]);
g1.set(xticks=ticksDIR);

From the logarithmic transformation above, we can see several plots past the ratio of 1, indicating there debts are greater then there available income.
It would interesting to see what the ProsperRatingNum as well as the Borrow Rate are for these individuals including there Income Range.

In [ ]:
# new dataframe
loans_DIR_filter = loans_df.copy()[loans_df['DebtToIncomeRatio']>1] # filter dataframe by DIR >1
loans_DIR_filter.sample(5)

In [ ]:
loans_DIR_filter.shape

In [ ]:
g2=sb.jointplot(data=loans_DIR_filter, x='BorrowerRate', y='DebtToIncomeRatio', hue='CreditRating', xlim=(0, 0.36), ylim=(-0.5, 10.4))
g2.set_axis_labels(xlabel='Borrow Rate %');

From the above jointplot, for the clients with a debt to income ratio greater then 1, we can see there are a mix of all types of rating Credit ratings. We can confirm that the variable has no effect on Borrower Rate as a result.

#### ListingCreationDate & ClosedDate

In [ ]:
loans_df.shape

In [ ]:
loans_df.ClosedDate.value_counts(dropna=False)
# There are a significant amount of NaT, i.e. loans not yet finished.

In [ ]:
# graph 1
g0 = sb.FacetGrid(data=loans_df, col='ListingCreationYear', 
                  col_wrap=3, # number of columns before starting a new row
                  height=3,
                  xlim = (0,0.38),
                  aspect=2.5, sharex=False, margin_titles=True);
g0.map_dataframe(sb.histplot, x='BorrowerRate');
g0.set(xticks=np.arange(0, 0.38, 0.02), xlabel='Borrow Rate %' );

g0.fig.subplots_adjust(top=0.9)
g0.fig.suptitle('Listing Creation Year', fontsize=20 );


# graph 2
g1 = sb.FacetGrid(data=loans_df, col='ClosedDateYear', 
                  col_wrap=3, # number of columns before starting a new row
                  height=3,
                  aspect=2.5, sharex=False, margin_titles=True);
g1.map_dataframe(sb.histplot, x='BorrowerRate');
g1.set(xticks=np.arange(0, 0.38, 0.02), xlabel='Borrow Rate %',
        ylabel='MonthlyLoanPayment ($)' );

g1.fig.subplots_adjust(top=0.9)
g1.fig.suptitle('Closed Date Year', fontsize=20);

From the above histogram plots faceted based on years, we can see that 2013 was the year with the most loans closed which makes sense for the following reasons:
- A majority of the loans taken had a term of 36 months/ 3 years which would be up for those who have opened a loan during 2011, which from the plot above indicates there was a significant up. 
- Loans taken out in 2009 would have elapsed and compounded with the above.
- The loans opened in 2012 have a very similar distribution, in particular the peak.